# Predictor y productor de música con IA

##Maxime Vilcocq Parra, A01710550
##Módulo 2 Implementación de un modelo de deep learning. (Portafolio Implementación)

Problema: A pesar de que siempre me ha gustado la música, no tengo ni la más remota idea de cómo funciona. Por ello, núnca podría crear una canción por mi cuenta.

En este trabajo, pretendo utilizar deeplearning para entrenar a un compositor de IA.

#Extracción

Dependencias

In [ ]:

!apt-get install -y fluidsynth
!pip install -q music21 pretty_midi midi2audio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0
  libinput-bin libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libx

music21: librería para analizar y manipular partituras MIDI.

pretty_midi + midi2audio: permiten convertir un .mid a WAV y escucharlo.

fluidsynth: sintetizador que reproduce el audio directamente en Colab.

In [ ]:
import os
import json
import urllib.request
import numpy as np
from pathlib import Path
from music21 import converter, instrument, note, chord, corpus

In [ ]:
# Carpeta base del proyecto
BASE_DIR = Path("/content/melody_project")
DATA_DIR = BASE_DIR / "data"

midi_clasico  = DATA_DIR / "midi_clasico"
midi_videojuegos = DATA_DIR / "midi_videojuegos"
midi_otro   = DATA_DIR / "midi_otro" #de momento no lo ocupo

# Crear las carpetas
for folder in [midi_clasico, midi_videojuegos, midi_otro]:
    folder.mkdir(parents=True, exist_ok=True)

print("Carpetas creadas:")
print(midi_clasico)
print(midi_videojuegos)
print(midi_otro)


Carpetas creadas:
/content/melody_project/data/midi_clasico
/content/melody_project/data/midi_videojuegos
/content/melody_project/data/midi_otro


##Descarga de música clásica con music21

In [ ]:
from music21 import corpus, converter
from pathlib import Path

midi_clasico = Path("melody_project/data/midi_clasico")
midi_clasico.mkdir(parents=True, exist_ok=True)

# Obtenemos listas de piezas por compositor
bach_piezas = corpus.getComposer('bach')
handel_piezas = corpus.getComposer('handel')
scarlatti_piezas = corpus.getComposer('scarlatti')

# Seleccionamos una cantidad razonable de cada uno (ajusta si quieres más)
bach_sample = bach_piezas[:100]       # Muchos corales, fugas, invenciones
handel_sample = handel_piezas[:100]   # Arias, suites, sonatas
scarlatti_sample = scarlatti_piezas[:100] # Sonatas cortas, gran variedad

# Combinamos todos
clasico_total = bach_sample + handel_sample + scarlatti_sample
print(f"Se usarán {len(clasico_total)} piezas clásicas en total")

# Guardamos los MIDI
for idx, pieza_path in enumerate(clasico_total):
    try:
        midi_stream = converter.parse(pieza_path)
        out_path = midi_clasico / f"clasico_{idx}.mid"
        midi_stream.write('midi', fp=out_path)
    except Exception as e:
        print(f"⚠️ Error al convertir {pieza_path}: {e}")

print("\nArchivos de música clásica listos en:")
!ls -lh {midi_clasico}



Se usarán 101 piezas clásicas en total

Archivos de música clásica listos en:
total 476K
-rw-r--r-- 1 root root 4.6K Nov  6 22:22 clasico_0.mid
-rw-r--r-- 1 root root 6.2K Nov  6 22:23 clasico_100.mid
-rw-r--r-- 1 root root 4.8K Nov  6 22:22 clasico_10.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_11.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_12.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_13.mid
-rw-r--r-- 1 root root 2.4K Nov  6 22:22 clasico_14.mid
-rw-r--r-- 1 root root 2.1K Nov  6 22:22 clasico_15.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_16.mid
-rw-r--r-- 1 root root 1.9K Nov  6 22:22 clasico_17.mid
-rw-r--r-- 1 root root 2.9K Nov  6 22:22 clasico_18.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_19.mid
-rw-r--r-- 1 root root 2.1K Nov  6 22:22 clasico_1.mid
-rw-r--r-- 1 root root 4.9K Nov  6 22:22 clasico_20.mid
-rw-r--r-- 1 root root 2.6K Nov  6 22:22 clasico_21.mid
-rw-r--r-- 1 root root 1.6K Nov  6 22:22 clasico_22.mid
-rw-r--r-- 1 roo

##Descarga de música externa manual

use la página de https://bitmidi.com/

In [ ]:
import requests

game_urls = {
    "super_mario.mid": "https://bitmidi.com/uploads/98137.mid",
    "wii.mid": "https://bitmidi.com/uploads/110325.mid",
    "animal_c1": "https://bitmidi.com/uploads/6835.mid",
    "animal_c2": "https://bitmidi.com/uploads/6836.mid",
    "animal_c3": "https://bitmidi.com/uploads/6837.mid",
    "animal_c4": "https://bitmidi.com/uploads/6838.mid",
    "animal_c5": "https://bitmidi.com/uploads/6839.mid",
    "animal_c6": "https://bitmidi.com/uploads/6840.mid",
    "animal_c7": "https://bitmidi.com/uploads/6841.mid",
    "animal_c8": "https://bitmidi.com/uploads/6842.mid",
    "animal_c9": "https://bitmidi.com/uploads/6843.mid",
    "animal_c10": "https://bitmidi.com/uploads/6844.mid",
    "animal_c11": "https://bitmidi.com/uploads/6845.mid",
    "animal_c12": "https://bitmidi.com/uploads/6846.mid",
    "animal_c13": "https://bitmidi.com/uploads/6847.mid",
    "animal_c14": "https://bitmidi.com/uploads/6848.mid",
    "animal_c15": "https://bitmidi.com/uploads/6849.mid",
    "animal_c16": "https://bitmidi.com/uploads/6825.mid",
    "animal_c17": "https://bitmidi.com/uploads/6826.mid",
    "animal_c18": "https://bitmidi.com/uploads/6827.mid",
    "animal_c19": "https://bitmidi.com/uploads/6828.mid",
    "animal_c20": "https://bitmidi.com/uploads/6829.mid",
    "animal_c21": "https://bitmidi.com/uploads/6830.mid",
    "animal_c22": "https://bitmidi.com/uploads/6831.mid",
    "animal_c23": "https://bitmidi.com/uploads/6832.mid",
    "animal_c24": "https://bitmidi.com/uploads/6833.mid",
    "animal_c25": "https://bitmidi.com/uploads/6834.mid",
    "zelda1.mid": "https://bitmidi.com/uploads/101253.mid",
    "zelda2.mid": "https://bitmidi.com/uploads/101446.mid",
    "zelda3.mid": "https://bitmidi.com/uploads/101568.mid",
    "zelda4.mid": "https://bitmidi.com/uploads/112951.mid",
    "zelda5.mid": "https://bitmidi.com/uploads/112952.mid",
    "zelda6.mid": "https://bitmidi.com/uploads/112953.mid",
    "zelda7.mid": "https://bitmidi.com/uploads/112954.mid",
    "zelda8.mid": "https://bitmidi.com/uploads/112955.mid",
    "zelda9.mid": "https://bitmidi.com/uploads/112987.mid",
    "zelda10.mid": "https://bitmidi.com/uploads/112988.mid",
    "zelda11.mid": "https://bitmidi.com/uploads/68139.mid",
    "zelda12.mid": "https://bitmidi.com/uploads/68140.mid",
    "zelda13.mid": "https://bitmidi.com/uploads/98428.mid",
    "zelda14.mid": "https://bitmidi.com/uploads/101241.mid",
    "zelda15.mid": "https://bitmidi.com/uploads/101244.mid",
    "zelda16.mid": "https://bitmidi.com/uploads/101246.mid",
    "zelda17.mid": "https://bitmidi.com/uploads/101249.mid",
    "zelda18.mid": "https://bitmidi.com/uploads/101254.mid",
    "zelda19.mid": "https://bitmidi.com/uploads/101249.mid",
    "mario_kart1": "https://bitmidi.com/uploads/72273.mid",
    "mario_kart2": "https://bitmidi.com/uploads/98265.mid",
    "mario_kart3": "https://bitmidi.com/uploads/72267.mid",
    "mario_kart4": "https://bitmidi.com/uploads/72268.mid",
    "mario_kart5": "https://bitmidi.com/uploads/72269.mid",
    "mario_kart6": "https://bitmidi.com/uploads/72270.mid",
    "mario_kart7": "https://bitmidi.com/uploads/72271.mid",
    "mario_kart8": "https://bitmidi.com/uploads/72272.mid",
    "mario_kart9": "https://bitmidi.com/uploads/72274.mid",
    "mario_kart10": "https://bitmidi.com/uploads/72275.mid",
    "mario_kart11": "https://bitmidi.com/uploads/72276.mid",
    "mario_kart12": "https://bitmidi.com/uploads/72281.mid",
    "mario_kart13": "https://bitmidi.com/uploads/72282.mid",
    "mario_kart14": "https://bitmidi.com/uploads/72283.mid",
    "mario_kart15": "https://bitmidi.com/uploads/72284.mid",
    "mario_kart16": "https://bitmidi.com/uploads/72285.mid",
    "mario_kart17": "https://bitmidi.com/uploads/72287.mid",
    "mario_kart18": "https://bitmidi.com/uploads/72288.mid",
    "mario_kart19": "https://bitmidi.com/uploads/72289.mid",
    "mario_kart20": "https://bitmidi.com/uploads/72290.mid",
    "mario_kart21": "https://bitmidi.com/uploads/72291.mid",
    "mario_kart22": "https://bitmidi.com/uploads/72293.mid",
    "mario_kart23": "https://bitmidi.com/uploads/72294.mid",
    "mario_kart24": "https://bitmidi.com/uploads/72295.mid",
    "mario_kart25": "https://bitmidi.com/uploads/72296.mid",
    "smash_bros1": "https://bitmidi.com/uploads/98421.mid",
    "smash_bros2": "https://bitmidi.com/uploads/98410.mid",
    "smash_bros3": "https://bitmidi.com/uploads/98411.mid",
    "smash_bros4": "https://bitmidi.com/uploads/98412.mid",
    "smash_bros5": "https://bitmidi.com/uploads/98413.mid",
    "smash_bros6": "https://bitmidi.com/uploads/98414.mid",
    "smash_bros7": "https://bitmidi.com/uploads/98415.mid",
    "smash_bros8": "https://bitmidi.com/uploads/98416.mid",
    "smash_bros9": "https://bitmidi.com/uploads/98417.mid",
    "smash_bros10": "https://bitmidi.com/uploads/98418.mid",
    "smash_bros11": "https://bitmidi.com/uploads/98419.mid",
    "smash_bros12": "https://bitmidi.com/uploads/98420.mid",
    "smash_bros13": "https://bitmidi.com/uploads/98423.mid",
    "smash_bros14": "https://bitmidi.com/uploads/98424.mid",
    "smash_bros15": "https://bitmidi.com/uploads/98426.mid",
    "smash_bros16": "https://bitmidi.com/uploads/98427.mid",
    "smash_bros17": "https://bitmidi.com/uploads/98429.mid",
    "smash_bros18": "https://bitmidi.com/uploads/98430.mid",
    "smash_bros19": "https://bitmidi.com/uploads/98432.mid",
    "smash_bros20": "https://bitmidi.com/uploads/98435.mid",
    "smash_bros21": "https://bitmidi.com/uploads/98436.mid",
    "smash_bros22": "https://bitmidi.com/uploads/98437.mid",
    "smash_bros23": "https://bitmidi.com/uploads/98440.mid",
    "smash_bros24": "https://bitmidi.com/uploads/98441.mid",
    "smash_bros25": "https://bitmidi.com/uploads/98444.mid",
    "sonic1": "https://bitmidi.com/uploads/95745.mid",
    "sonic2": "https://bitmidi.com/uploads/95713.mid",
    "sonic3": "https://bitmidi.com/uploads/95715.mid",
    "sonic4": "https://bitmidi.com/uploads/95719.mid",
    "sonic5": "https://bitmidi.com/uploads/95720.mid",
}

headers = {"User-Agent": "Mozilla/5.0"}

for fname, url in game_urls.items():
    out_path = midi_videojuegos / fname
    print(f"Descargando {fname} desde {url}")
    try:
        r = requests.get(url, headers=headers, timeout=20)
        r.raise_for_status()  # genera excepción si hay error
        with open(out_path, "wb") as f:
            f.write(r.content)
        print(f"Guardado: {fname}")
    except Exception as e:
        print(f"Error descargando {fname}: {e}")

print("\nArchivos de videojuegos listos en:")
!ls -lh {midi_videojuegos}


Descargando super_mario.mid desde https://bitmidi.com/uploads/98137.mid
Guardado: super_mario.mid
Descargando wii.mid desde https://bitmidi.com/uploads/110325.mid
Guardado: wii.mid
Descargando animal_c1 desde https://bitmidi.com/uploads/6835.mid
Guardado: animal_c1
Descargando animal_c2 desde https://bitmidi.com/uploads/6836.mid
Guardado: animal_c2
Descargando animal_c3 desde https://bitmidi.com/uploads/6837.mid
Guardado: animal_c3
Descargando animal_c4 desde https://bitmidi.com/uploads/6838.mid
Guardado: animal_c4
Descargando animal_c5 desde https://bitmidi.com/uploads/6839.mid
Guardado: animal_c5
Descargando animal_c6 desde https://bitmidi.com/uploads/6840.mid
Guardado: animal_c6
Descargando animal_c7 desde https://bitmidi.com/uploads/6841.mid
Guardado: animal_c7
Descargando animal_c8 desde https://bitmidi.com/uploads/6842.mid
Guardado: animal_c8
Descargando animal_c9 desde https://bitmidi.com/uploads/6843.mid
Guardado: animal_c9
Descargando animal_c10 desde https://bitmidi.com/uploa

In [ ]:
!ls -lh {midi_clasico}
!ls -lh {midi_videojuegos}


total 476K
-rw-r--r-- 1 root root 4.6K Nov  6 22:22 clasico_0.mid
-rw-r--r-- 1 root root 6.2K Nov  6 22:23 clasico_100.mid
-rw-r--r-- 1 root root 4.8K Nov  6 22:22 clasico_10.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_11.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_12.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_13.mid
-rw-r--r-- 1 root root 2.4K Nov  6 22:22 clasico_14.mid
-rw-r--r-- 1 root root 2.1K Nov  6 22:22 clasico_15.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_16.mid
-rw-r--r-- 1 root root 1.9K Nov  6 22:22 clasico_17.mid
-rw-r--r-- 1 root root 2.9K Nov  6 22:22 clasico_18.mid
-rw-r--r-- 1 root root 2.8K Nov  6 22:22 clasico_19.mid
-rw-r--r-- 1 root root 2.1K Nov  6 22:22 clasico_1.mid
-rw-r--r-- 1 root root 4.9K Nov  6 22:22 clasico_20.mid
-rw-r--r-- 1 root root 2.6K Nov  6 22:22 clasico_21.mid
-rw-r--r-- 1 root root 1.6K Nov  6 22:22 clasico_22.mid
-rw-r--r-- 1 root root 2.9K Nov  6 22:22 clasico_23.mid
-rw-r--r-- 1 root root 3.1K Nov  6 22:

Extraer las notas

In [ ]:
#Extrae notas y acordes de todos los archivos MIDI en una carpeta
def extract_notes(midi_folder):

    notes = []
    for file in os.listdir(midi_folder):
        if not file.endswith(".mid"):
            continue
        fpath = midi_folder / file
        if fpath.stat().st_size == 0:
            print(f"Archivo vacío: {file}, se omite.")
            continue

        print("Procesando:", file)
        try:
            midi = converter.parse(fpath)
            parts = instrument.partitionByInstrument(midi)
            elements = parts.parts[0].recurse() if parts else midi.flat.notes
        except Exception as e:
            print(f"Error procesando {file}: {e}")
            continue

        for element in elements:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n.pitch) for n in element.notes))
    return notes


# Extracción por carpeta
notas_clasico = extract_notes(midi_clasico)
notas_videojuegos = extract_notes(midi_videojuegos)

# Combinar notas
notas_combinadas = notas_clasico + notas_videojuegos
print("Total de notas extraídas:", len(notas_combinadas))


Procesando: clasico_36.mid
Procesando: clasico_15.mid
Procesando: clasico_8.mid
Procesando: clasico_52.mid
Procesando: clasico_84.mid
Procesando: clasico_83.mid
Procesando: clasico_16.mid
Procesando: clasico_74.mid
Procesando: clasico_24.mid
Procesando: clasico_6.mid
Procesando: clasico_91.mid
Procesando: clasico_93.mid
Procesando: clasico_41.mid
Procesando: clasico_66.mid
Procesando: clasico_30.mid
Procesando: clasico_2.mid
Procesando: clasico_1.mid
Procesando: clasico_47.mid
Procesando: clasico_82.mid
Procesando: clasico_89.mid
Procesando: clasico_19.mid
Procesando: clasico_39.mid
Procesando: clasico_46.mid
Procesando: clasico_27.mid
Procesando: clasico_21.mid
Procesando: clasico_10.mid
Procesando: clasico_51.mid
Procesando: clasico_79.mid
Procesando: clasico_60.mid
Procesando: clasico_61.mid
Procesando: clasico_55.mid
Procesando: clasico_88.mid
Procesando: clasico_13.mid
Procesando: clasico_99.mid
Procesando: clasico_28.mid
Procesando: clasico_65.mid
Procesando: clasico_71.mid
Proce

Convertir las notas a algo interpretable (Procesar los datos)

In [ ]:
def process_and_save(notes, prefix, output_dir):
    unique_notes = sorted(set(notes))
    note_to_int = {n: i for i, n in enumerate(unique_notes)}
    encoded = [note_to_int[n] for n in notes]

    SEQ_LENGTH = 50
    inputs, targets = [], []
    for i in range(len(encoded) - SEQ_LENGTH):
        inputs.append(encoded[i:i+SEQ_LENGTH])
        targets.append(encoded[i+SEQ_LENGTH])

    inputs = np.array(inputs)
    targets = np.array(targets)

    np.save(output_dir / f"{prefix}_notas.npy", np.array(notes))
    np.savez(output_dir / f"{prefix}_secuencias_datos.npz", inputs=inputs, targets=targets)
    json.dump(unique_notes, open(output_dir / f"{prefix}_notas_unicas.json", "w"))

    print(f"Dataset '{prefix}' guardado: {inputs.shape[0]} secuencias, {len(unique_notes)} notas únicas")

In [ ]:
process_and_save(notas_clasico, "clasico", DATA_DIR)
process_and_save(notas_videojuegos, "videojuegos", DATA_DIR)
process_and_save(notas_combinadas, "combinado", DATA_DIR)

Dataset 'clasico' guardado: 1143 secuencias, 46 notas únicas
Dataset 'videojuegos' guardado: 3700 secuencias, 354 notas únicas
Dataset 'combinado' guardado: 4893 secuencias, 357 notas únicas


In [ ]:
print("\nArchivos generados:")
!ls -lh {DATA_DIR}


Archivos generados:
total 4.5M
-rw-r--r-- 1 root root  24K Nov  6 22:23 clasico_notas.npy
-rw-r--r-- 1 root root  300 Nov  6 22:23 clasico_notas_unicas.json
-rw-r--r-- 1 root root 456K Nov  6 22:23 clasico_secuencias_datos.npz
-rw-r--r-- 1 root root 329K Nov  6 22:23 combinado_notas.npy
-rw-r--r-- 1 root root 3.7K Nov  6 22:23 combinado_notas_unicas.json
-rw-r--r-- 1 root root 2.0M Nov  6 22:23 combinado_secuencias_datos.npz
drwxr-xr-x 2 root root 4.0K Nov  6 22:23 midi_clasico
drwxr-xr-x 2 root root 4.0K Nov  6 22:22 midi_otro
drwxr-xr-x 2 root root 4.0K Nov  6 22:23 midi_videojuegos
-rw-r--r-- 1 root root 250K Nov  6 22:23 videojuegos_notas.npy
-rw-r--r-- 1 root root 3.6K Nov  6 22:23 videojuegos_notas_unicas.json
-rw-r--r-- 1 root root 1.5M Nov  6 22:23 videojuegos_secuencias_datos.npz


En este último bloque de código hemos generado 3 archivos importantes por cada género:

**notas.npy**:	Lista completa de las notas extraídas (texto, ej. "C4", "E4.G4")

**secuencias_datos.npz**: Inputs y targets para entrenar la red

**notas_unicas.json**: Diccionario de las notas únicas para volver a convertir nuestras predicciones a notas.

Descargar esto para utilizar en el colab de entrenamiento del modelo: